<a href="https://colab.research.google.com/github/peremartra/Tailoring-LLM-Architectures/blob/main/APPC/APPC_NB01_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tailoring LLM Architectures
## Surgical Optimization Beyond Fine-Tuning


### Appendix C: Energy consumption evaluation
### Notebook: 01. How to use `measure_energy_consumption`
by [Pere Martra](https://github.com/peremartra)

[![LinkedIn](https://img.shields.io/badge/LinkedIn-0077B5?style=flat&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/pere-martra/) [![GitHub](https://img.shields.io/badge/GitHub-100000?style=flat&logo=github&logoColor=white)](https://github.com/peremartra) [![X](https://img.shields.io/badge/X-000000?style=flat&logo=x&logoColor=white)](https://x.com/PereMartra) [![Hugging Face](https://img.shields.io/badge/🤗%20Hugging%20Face-blue)](https://huggingface.co/oopere)

_____
Colab Environment: GPU T4

Models:
* gemma-3-270m
_____


In [1]:
# Install dependencies
!pip install -q lm-eval transformers torch accelerate codecarbon

# Import libraries
import torch, gc, time
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from codecarbon import EmissionsTracker
from torch.utils.data import DataLoader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.6/357.6 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━

In [2]:
# Load model and tokenizer
model_name = "google/gemma-3-270m"
print(f"Loading model: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

print(f"Model loaded on device: {model.device}")


Loading model: google/gemma-3-270m


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]

Model loaded on device: cuda:0


In [3]:
RECOVERY_SAMPLES = 100
MAX_LENGTH = 1024
BATCH_SIZE = 4
MAX_SAMPLES_CARBON=50

In [4]:
def prepare_dataset(dataset, text_field='text'):
    """
    Tokenizes and prepares a dataset for calibration.

    Handles different dataset formats (WikiText uses 'text', SMS uses 'sms' field).
    """
    def tokenize_function(examples):
        if text_field in examples:
            texts = examples[text_field]
        elif 'sms' in examples:  # SMS dataset specific
            texts = examples['sms']
        elif 'text' in examples:
            texts = examples['text']
        else:
            texts = examples[list(examples.keys())[0]]

        return tokenizer(
            texts,
            truncation=True,
            padding='max_length',
            max_length=MAX_LENGTH,
            return_tensors='pt'
        )

    tokenized = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)
    tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask'])
    return DataLoader(tokenized, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
datawiki = load_dataset('wikitext', 'wikitext-2-raw-v1', split=f'train[:{RECOVERY_SAMPLES}]')
dataloaderwiki = prepare_dataset(datawiki)  # WikiText (largo)

README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
def clear_gpu_cache():
    """Clear GPU cache completely"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    gc.collect()

### Listing C.1 Calibrating GPU idle power


In [7]:
def calibrate_idle_power(device="cuda", duration_seconds=30):
    clear_gpu_cache()  #A


    tracker = EmissionsTracker(
        project_name="idle_calibration",
        measure_power_secs=1,  #B
        save_to_file=False,
        log_level="error"
    )
    tracker.start()
    start_time = time.time()
    time.sleep(duration_seconds)  #C
    emissions_data = tracker.stop()
    actual_duration = time.time() - start_time

    energy_kwh = tracker._total_energy.kWh #D
    idle_power_watts = (energy_kwh * 1000) / (actual_duration / 3600) #E

    gpu_temp = None
    if torch.cuda.is_available():
        try:
            gpu_temp = torch.cuda.temperature()
        except:
            gpu_temp = None

    if gpu_temp is not None:
        print(f"   GPU Temperature: {gpu_temp:.1f}°C")

    return {
        "idle_power_watts": idle_power_watts,
        "gpu_temp_celsius": gpu_temp,
    }


In [8]:
idle_calibration = calibrate_idle_power(device="cuda", duration_seconds=30)
idle_watts = idle_calibration["idle_power_watts"]

[codecarbon WARNING @ 16:01:26] Multiple instances of codecarbon are allowed to run at the same time.


   GPU Temperature: 47.0°C


### Listing C.2 Measuring net energy consumption


In [9]:
def measure_energy_consumption(model, tokenizer, data_source,
                               idle_power_watts, max_samples=50,
                               max_new_tokens=50):
    model.eval()

    samples = []
    for batch in data_source:
        for i in range(len(batch['input_ids'])):
            samples.append(batch['input_ids'][i])
            if len(samples) >= max_samples:
                break
        if len(samples) >= max_samples:
            break  #A

    warmup_input = samples[0].unsqueeze(0).to(model.device)
    with torch.no_grad():
        for _ in range(2):
            model.generate(
                warmup_input,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id
            )
    torch.cuda.synchronize()  #B

    tracker = EmissionsTracker(
        measure_power_secs=1,
        save_to_file=False,
        log_level="error"
    )  #C

    total_tokens_generated = 0
    tracker.start()
    start_time = time.time()

    with torch.no_grad():
        for sample in samples:
            input_ids = sample.unsqueeze(0).to(model.device)
            torch.cuda.synchronize()  #D

            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id
            )

            torch.cuda.synchronize()

            num_new_tokens = outputs.shape[1] - input_ids.shape[1]
            total_tokens_generated += num_new_tokens

    tracker.stop()
    duration_seconds = time.time() - start_time
    emissions_raw_kwh = tracker._total_energy.kWh  #E

    idle_energy_kwh = (idle_power_watts * duration_seconds) / 3_600_000
    energy_net_kwh = max(0.0, emissions_raw_kwh - idle_energy_kwh)  #F

    total_joules_net = energy_net_kwh * 3_600_000
    joules_per_token = (total_joules_net / total_tokens_generated
                       if total_tokens_generated > 0 else 0.0)  #G

    return {
        "energy_net_kwh": float(energy_net_kwh),
        "efficiency_joules_per_token": float(joules_per_token)
    }


In [10]:
metrics_base_carbon = measure_energy_consumption(model, tokenizer, dataloaderwiki,
                                                 idle_power_watts=idle_watts,
                                                 max_samples=MAX_SAMPLES_CARBON, max_new_tokens=50)

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [11]:
metrics_base_carbon

{'energy_net_kwh': 0.0001813826314681858,
 'efficiency_joules_per_token': 0.26119098931418755}